In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(100_000_000_000)

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils
from utils import *

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:111: UserWarning: The Shapely GEOS version (3.9.1-CAPI-1.14.2) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
GCS_FILE_PATH

'gs://calitp-analytics-data/data-analyses/rt_delay/'

In [4]:
old = pd.read_parquet(f'{GCS_FILE_PATH}lbt_positions.parquet')
old.head(3)

,calitp_itp_id,calitp_url_number,entity_id,vehicle_id,vehicle_trip_id,vehicle_timestamp,vehicle_position_latitude,vehicle_position_longitude,header_timestamp
0,170,0,2403,None,9288416,2021-11-29 15:59:12,33.775260,-118.152000,2021-11-29 16:00:04
1,170,0,2404,None,9288666,2021-11-29 15:59:17,33.768030,-118.191220,2021-11-29 16:00:04
2,170,0,2407,None,9288521,2021-11-29 15:59:33,33.775394,-118.141396,2021-11-29 16:00:04


In [5]:
new = query_sql('''
SELECT * FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE calitp_itp_id = 170 LIMIT 1000
''')

In [6]:
new.head(3)

,calitp_itp_id,calitp_url_number,calitp_filepath,id,header,vehicle
0,170,0,/tmp/tmp8kb4cpoy/2022-02-09T19:45:48/170/0/gtf...,2403,"{'timestamp': 1644435943, 'incrementality': No...","{'vehicle': {'licensePlate': None, 'label': '2..."
1,170,0,/tmp/tmp8kb4cpoy/2022-02-09T19:45:48/170/0/gtf...,2404,"{'timestamp': 1644435943, 'incrementality': No...","{'vehicle': {'licensePlate': None, 'label': '2..."
2,170,0,/tmp/tmp8kb4cpoy/2022-02-09T19:45:48/170/0/gtf...,2407,"{'timestamp': 1644435943, 'incrementality': No...","{'vehicle': {'licensePlate': None, 'label': '2..."


In [12]:
new.vehicle.iloc[0]

{'vehicle': {'licensePlate': None, 'label': '2403', 'id': None},
 'trip': {'tripId': '9370849',
  'routeId': None,
  'directionId': None,
  'startTime': None,
  'startDate': None,
  'scheduleRelationship': None},
 'position': {'latitude': 33.760986,
  'longitude': -118.15106,
  'bearing': None,
  'odometer': None,
  'speed': None},
 'currentStopSequence': None,
 'stopId': None,
 'currentStatus': None,
 'timestamp': 1644435845,
 'congestionLevel': None,
 'occupancyStatus': None,
 'occupancyPercentage': None}

In [36]:
%%sql -o new
SELECT calitp_itp_id, calitp_url_number,
header.timestamp AS header_timestamp, vehicle.timestamp AS vehicle_timestamp,
vehicle.vehicle.label AS entity_id, vehicle.vehicle.id AS vehicle_id,
vehicle.trip.tripId AS vehicle_trip_id, vehicle.position.longitude AS vehicle_longitude,
vehicle.position.latitude AS vehicle_latitude
FROM `cal-itp-data-infra.gtfs_rt.vehicle_positions`
WHERE calitp_itp_id = 170 AND vehicle.timestamp > 1644393600 AND vehicle.timestamp < 1644487200

,calitp_itp_id,calitp_url_number,header_timestamp,vehicle_timestamp,entity_id,vehicle_id,vehicle_trip_id,vehicle_longitude,vehicle_latitude
0,170,0,1644435943,1644435845,2403,None,9370849,-118.151060,33.760986
1,170,0,1644435943,1644435872,2404,None,9370203,-118.154660,33.771730
2,170,0,1644435943,1644435874,2407,None,9369934,-118.120186,33.775400
3,170,0,1644435943,1644435911,2409,None,9370154,-118.107640,33.847557
4,170,0,1644435943,1644435870,2410,None,9370132,-118.100020,33.887980
...,...,...,...,...,...,...,...,...,...
237147,170,0,1644435943,1644435889,1831,None,9370693,-118.189520,33.800274
237148,170,0,1644435943,1644435884,1832,None,9369750,-118.184870,33.855206
237149,170,0,1644435943,1644435880,1833,None,9369643,-118.215546,33.797770
237150,170,0,1644435943,1644435890,1838,None,9371122,-118.196754,33.782665


In [37]:
new.vehicle_timestamp = new.vehicle_timestamp.apply(convert_ts)

In [39]:
new.vehicle_timestamp.max()

Timestamp('2022-02-09 15:58:45')

In [40]:
new.vehicle_timestamp.min()

Timestamp('2022-02-09 04:15:51')

In [30]:
analysis_date = dt.date(2022, 2, 9)

In [31]:
start = dt.datetime.combine(analysis_date, dt.time(0))
end = start + dt.timedelta(days = 1, seconds = 2 * 60**2)

In [32]:
start

datetime.datetime(2022, 2, 9, 0, 0)

In [33]:
end

datetime.datetime(2022, 2, 10, 2, 0)

In [34]:
start.timestamp()

1644393600.0

In [35]:
end.timestamp()

1644487200.0